# 📧 Support Email Copilot — Microsoft Agent Framework

**A complete learning journey:** Build an AI-powered support email system from scratch, progressively adding capabilities.

---

## 📚 Table of Contents

| # | Section | What You'll Learn |
|---|---------|------------------|
| **0** | [Shared Setup](#0-shared-setup) | Environment, models, sample data |
| **1** | [Basic Agent](#1-basic-agent) | Create and run your first agent |
| **2** | [Streaming](#2-streaming-responses) | Real-time token streaming |
| **3** | [Multi-Turn Conversations](#3-multi-turn-conversations) | Thread-based memory |
| **4** | [Function Tools](#4-function-tools) | Add custom capabilities |
| **5** | [Human-in-the-Loop](#5-human-in-the-loop-approval) | Approval workflows |
| **6** | [Middleware](#6-middleware) | Logging & observability |
| **7** | [Memory](#7-agent-memory) | Persistent user context |
| **8** | [Sequential Workflows](#8-sequential-workflows) | Classify → Draft → Review |
| **9** | [Branching Logic](#9-branching-logic) | Spam vs. NotSpam vs. Uncertain |
| **10** | [Fan-Out/Fan-In](#10-fan-out--fan-in) | Parallel processing |
| **11** | [Multi-Agent Group Chat](#11-multi-agent-group-chat) | Team collaboration |
| **12** | [Capstone Demo](#12-capstone-demo) | End-to-end system |

---

## 🎯 What We Will Build By The End

By completing this notebook, you'll have built a **Support Email Copilot** that:

- ✅ **Classifies** incoming emails (Spam / Not Spam / Uncertain)
- ✅ **Looks up** customer SLA and ticket status via function tools
- ✅ **Drafts** professional responses with customizable tone
- ✅ **Requires approval** before sending sensitive replies
- ✅ **Remembers** user preferences (language, tone, name)
- ✅ **Processes in parallel** for long emails (response + summary)
- ✅ **Uses multiple reviewers** for quality control (security, tone, accuracy)
- ✅ **Logs** every operation for observability

---

## Prerequisites

Before running this notebook:

1. ✅ **Azure subscription** with access to Azure OpenAI
2. ✅ **Azure OpenAI resource** with a deployed model (e.g., `gpt-4o-mini`)
3. ✅ **Azure CLI** installed and authenticated (`az login`)
4. ✅ **`.env` file** with your configuration

# 0. Shared Setup

> **Why this matters:** A consistent foundation ensures all examples work together and reduces code duplication.

## Install Python Packages

```bash
pip3.10 install agent-framework --pre python-dotenv nest_asyncio
```

In [ ]:
# Skip this cell - packages already installed in the .venv
%pip install agent-framework --pre python-dotenv nest_asyncio

## Load Environment & Create Chat Client

The `.env` file contains your Azure OpenAI configuration. We create **one** `chat_client` instance to reuse throughout the notebook.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables
load_dotenv()

# Create ONE chat client - reused throughout the notebook
chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

print("✅ Environment loaded and chat_client created")

## Shared Pydantic Models

These models are used consistently across all sections for structured input/output.

In [ ]:
from typing import Literal, Annotated
from pydantic import BaseModel, Field

# === Input Model ===
class EmailInput(BaseModel):
    """Incoming support email."""
    sender: str = Field(description="Email sender address")
    subject: str = Field(description="Email subject line")
    body: str = Field(description="Email body content")
    customer_id: str | None = Field(default=None, description="Customer ID if known")
    ticket_id: str | None = Field(default=None, description="Related ticket ID if any")

# === Classification Model ===
class ClassificationResult(BaseModel):
    """Result of email classification."""
    category: Literal["spam", "not_spam", "uncertain"] = Field(description="Email category")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence score 0-1")
    reason: str = Field(description="Brief explanation of classification")

# === Draft Response Model ===
class DraftResponse(BaseModel):
    """Draft reply to customer email."""
    subject: str = Field(description="Reply subject line")
    body: str = Field(description="Reply body")
    tone: Literal["formal", "friendly", "apologetic"] = Field(description="Tone used")
    needs_review: bool = Field(default=False, description="Flag if needs human review")

# === Final Response Model ===
class FinalResponse(BaseModel):
    """Final approved response."""
    classification: ClassificationResult
    draft: DraftResponse | None = Field(default=None, description="Draft if not spam")
    review_notes: str | None = Field(default=None, description="Reviewer comments")
    approved: bool = Field(default=False, description="Whether approved to send")

print("✅ Shared models defined: EmailInput, ClassificationResult, DraftResponse, FinalResponse")

## Sample Emails

Two example emails we'll use throughout the notebook — one legitimate support request, one spam.

In [ ]:
# === LEGITIMATE EMAIL ===
LEGIT_EMAIL = EmailInput(
    sender="sarah.chen@acmecorp.com",
    subject="Order #12345 - Delivery Issue",
    body="""Hi Support Team,

I placed order #12345 last week and the tracking shows it was delivered, 
but I never received the package. I've checked with my neighbors and the building 
concierge, but no one has seen it.

This is urgent as the items were needed for a client presentation on Friday.
Can you please help me locate the package or arrange a replacement?

Thank you,
Sarah Chen
Account: ACME-7891
""",
    customer_id="CUST-7891",
    ticket_id="TKT-2024-001"
)

# === SPAM EMAIL ===
SPAM_EMAIL = EmailInput(
    sender="winner@prize-notifications.biz",
    subject="🎉 CONGRATULATIONS! You've WON $1,000,000!!!",
    body="""URGENT NOTIFICATION!!!

You have been selected as the WINNER of our international lottery!
To claim your $1,000,000 prize, simply send your bank details and 
a processing fee of $500 to unlock your winnings.

ACT NOW - This offer expires in 24 HOURS!!!

Click here to claim: http://totally-legit-prize.com/claim
""",
    customer_id=None,
    ticket_id=None
)

# === AMBIGUOUS EMAIL ===
AMBIGUOUS_EMAIL = EmailInput(
    sender="j.smith@unknown-domain.net",
    subject="Partnership Opportunity",
    body="""Hello,

I found your company online and I'm interested in discussing a potential 
business partnership. We have a new product line that might complement your services.

Can we schedule a call this week?

Best,
J. Smith
""",
    customer_id=None,
    ticket_id=None
)

print("✅ Sample emails defined: LEGIT_EMAIL, SPAM_EMAIL, AMBIGUOUS_EMAIL")

# 1. Basic Agent

> **Why this matters:** The agent is the core building block — understanding how to create and run one is essential for everything that follows.

Create a support agent using `chat_client.as_agent()` with instructions for handling customer emails.

In [ ]:
# Create the core Support Agent - we'll enhance this throughout the notebook
support_agent = chat_client.as_agent(
    name="SupportAgent",
    instructions="""You are a helpful customer support agent for an e-commerce company.
Your job is to:
1. Understand customer issues from their emails
2. Draft professional, empathetic responses
3. Provide clear next steps when possible

Always be polite, acknowledge the customer's frustration, and offer concrete solutions."""
)

print("✅ support_agent created")

## Run the Agent

Call `agent.run()` with the email content. The result's `.text` property contains the response.

In [ ]:
# Run the support agent on our legitimate email
async def run_basic_agent():
    prompt = f"""Please draft a response to this customer email:

From: {LEGIT_EMAIL.sender}
Subject: {LEGIT_EMAIL.subject}

{LEGIT_EMAIL.body}
"""
    result = await support_agent.run(prompt)
    print("📧 Draft Response:\n")
    print(result.text)

asyncio.run(run_basic_agent())

# 2. Streaming Responses

> **Why this matters:** Streaming provides better UX by showing partial responses as they generate, crucial for customer-facing applications.

In [ ]:
### Stream the response token by token using the SAME support_agent
async def stream_support_response():
    prompt = f"""Please draft a response to this customer email:

From: {LEGIT_EMAIL.sender}
Subject: {LEGIT_EMAIL.subject}

{LEGIT_EMAIL.body}
"""
    print("📧 Streaming Draft Response:\n")
    async for update in support_agent.run_stream(prompt):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming

asyncio.run(stream_support_response())

# 3. Multi-Turn Conversations

> **Why this matters:** Support interactions often require back-and-forth clarification. Threads maintain context across multiple exchanges.

## Create a Thread

Agents are **stateless** — use `get_new_thread()` to maintain conversation history:

In [ ]:
# Create a thread for multi-turn conversation
thread = support_agent.get_new_thread()

# Turn 1: Summarize the customer issue
print("Turn 1: Summarize the issue")
print("-" * 50)
result1 = await support_agent.run(
    f"Summarize the key issues in this email in 2-3 bullet points:\n\n{LEGIT_EMAIL.body}", 
    thread=thread
)
print(result1.text)
print()

# Turn 2: Draft a response (agent remembers the summary from Turn 1)
print("Turn 2: Draft response with professional tone")
print("-" * 50)
result2 = await support_agent.run(
    "Now draft a professional response addressing each of those issues. Use a formal but empathetic tone.",
    thread=thread
)
print(result2.text)

# 4. Function Tools

> **Why this matters:** Real support agents need to look up customer data, check order status, and access internal systems. Tools make this possible.

## Define Support Tools

Create tools the agent can call to look up customer information:

In [ ]:
from agent_framework import tool
# Simulated database of customer SLAs
CUSTOMER_SLAS = {
    "CUST-7891": {"tier": "Premium", "response_time": "4 hours", "replacement_policy": "Free expedited replacement"},
    "CUST-1234": {"tier": "Standard", "response_time": "24 hours", "replacement_policy": "Standard replacement"},
}

# Simulated ticket database
TICKET_STATUSES = {
    "TKT-2024-001": {"status": "Open", "priority": "High", "assigned_to": "Support Team", "last_update": "2024-01-15"},
    "TKT-2024-002": {"status": "Resolved", "priority": "Low", "assigned_to": "Bot", "last_update": "2024-01-10"},
}

@tool(name="lookup_customer_sla", description="Look up a customer's SLA tier and policies")
def lookup_customer_sla(
    customer_id: Annotated[str, Field(description="The customer ID to look up (e.g., CUST-7891)")]
) -> str:
    """Look up customer SLA information."""
    if customer_id in CUSTOMER_SLAS:
        sla = CUSTOMER_SLAS[customer_id]
        return f"Customer {customer_id}: {sla['tier']} tier, {sla['response_time']} response time, {sla['replacement_policy']}"
    return f"Customer {customer_id} not found in system."

@tool(name="get_incident_status", description="Get the current status of a support ticket")
def get_incident_status(
    ticket_id: Annotated[str, Field(description="The ticket ID to check (e.g., TKT-2024-001)")]
) -> str:
    """Get ticket status information."""
    if ticket_id in TICKET_STATUSES:
        ticket = TICKET_STATUSES[ticket_id]
        return f"Ticket {ticket_id}: Status={ticket['status']}, Priority={ticket['priority']}, Assigned to={ticket['assigned_to']}, Last update={ticket['last_update']}"
    return f"Ticket {ticket_id} not found in system."

print("✅ Support tools defined: lookup_customer_sla, get_incident_status")

# Create Agent with Tools

Pass tools to the agent so it can look up information when needed:

In [ ]:
# Create support agent with tools
support_agent_with_tools = chat_client.as_agent(
    name="SupportAgentWithTools",
    instructions="""You are a customer support agent with access to internal systems.
When handling emails:
1. Look up the customer's SLA tier to understand their service level
2. Check ticket status if a ticket ID is mentioned
3. Use this information to provide appropriate responses and set expectations

Always be empathetic and use the customer's SLA tier to guide your response (e.g., Premium customers get expedited service).""",
    tools=[lookup_customer_sla, get_incident_status]
)

print("✅ support_agent_with_tools created")

## Test Tool Usage

The agent will automatically decide when to call tools based on the email content:

In [ ]:
# Test with the legitimate email that has customer_id and ticket_id
prompt = f"""Handle this customer support email. Look up their SLA and ticket status first:

From: {LEGIT_EMAIL.sender}
Subject: {LEGIT_EMAIL.subject}
Customer ID: {LEGIT_EMAIL.customer_id}
Ticket ID: {LEGIT_EMAIL.ticket_id}

{LEGIT_EMAIL.body}
"""

result = await support_agent_with_tools.run(prompt)
print("📧 Response (with tool lookups):\n")
print(result.text)

# 5. Human-in-the-Loop Approval

> **Why this matters:** Some actions (like sending emails, issuing refunds) need human approval before execution to prevent errors.

## Define Approval-Required Tool

Use `approval_mode="always_require"` for sensitive actions:

In [ ]:
from agent_framework import ChatMessage, Content, Role

# Tool that requires human approval before sending
@tool(approval_mode="always_require")
def send_email_reply(
    to: Annotated[str, Field(description="Recipient email address")],
    subject: Annotated[str, Field(description="Email subject")],
    body: Annotated[str, Field(description="Email body content")]
) -> str:
    """Send an email reply to the customer. Requires human approval."""
    # In production, this would actually send the email
    return f"✅ Email sent to {to} with subject '{subject}'"

# Create agent with the approval-required tool
approval_agent = chat_client.as_agent(
    name="ApprovalSupportAgent",
    instructions="""You are a customer support agent. After drafting a response, 
use the send_email_reply tool to send it. This will require human approval.""",
    tools=[lookup_customer_sla, get_incident_status, send_email_reply]
)

print("✅ approval_agent created with send_email_reply tool")

## Request and Check for Approval

When the agent tries to call an approval-required tool, it returns `user_input_requests` instead of executing:

In [ ]:
# Ask the agent to handle and send a response
prompt = f"""Handle this email and send a response:

From: {LEGIT_EMAIL.sender}
Subject: {LEGIT_EMAIL.subject}
Customer ID: {LEGIT_EMAIL.customer_id}

{LEGIT_EMAIL.body}
"""

result = await approval_agent.run(prompt)

# Check if approval is needed
if result.user_input_requests:
    print("🔒 APPROVAL REQUIRED!")
    for user_input_needed in result.user_input_requests:
        print(f"  Function: {user_input_needed.function_call.name}")
        print(f"  Arguments: {user_input_needed.function_call.arguments}")
else:
    print(result.text)

## Provide Approval and Continue

Use `to_function_approval_response(True/False)` to approve or reject:

In [ ]:
print("\n--- Handling Approval ---\n")
# Provide approval and continue the conversation
if result.user_input_requests:
    user_input_needed = result.user_input_requests[0]
    
    # Simulate human approval (in production, this would be interactive)
    user_approval = True
    print(f"✅ Human approved: {user_approval}\n")
    
    # Create approval response message
    approval_message = ChatMessage(
        role=Role.USER,
        contents=[user_input_needed.to_function_approval_response(user_approval)]
    )
    
    # Continue with approval
    final_result = await approval_agent.run([
        prompt,
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])
    print(f"📊 Final Result:\n{final_result.text}")

# 6. Middleware

> **Why this matters:** Production systems need logging, monitoring, and observability. Middleware intercepts agent execution without modifying core logic.

## Create Logging Middleware

Middleware receives context and a `next` function to continue execution:

In [ ]:
from typing import Callable, Awaitable
from agent_framework import AgentRunContext, FunctionInvocationContext
import time

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Log agent execution with timing."""
    print(f"🚀 Agent starting... ({len(context.messages)} message(s))")
    start_time = time.time()
    
    await next(context)  # Continue to agent execution
    
    elapsed = time.time() - start_time
    print(f"✅ Agent finished in {elapsed:.2f}s")

async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Log function tool calls."""
    print(f"  📞 Calling: {context.function.name}({context.arguments})")
    
    await next(context)
    
    print(f"  📤 Result: {context.result[:100]}..." if len(str(context.result)) > 100 else f"  📤 Result: {context.result}")

print("✅ Middleware defined: logging_agent_middleware, logging_function_middleware")

## Apply Middleware to Agent

Pass middleware when creating the agent:

In [ ]:
# Create agent with middleware for logging
middleware_agent = chat_client.as_agent(
    name="LoggingSupportAgent",
    instructions="You are a support agent. Look up customer information when handling requests.",
    tools=[lookup_customer_sla, get_incident_status],
    middleware=[logging_agent_middleware, logging_function_middleware]
)

# Test - you'll see logs for agent and function calls
prompt = f"Check the SLA for customer {LEGIT_EMAIL.customer_id} and ticket status for {LEGIT_EMAIL.ticket_id}"
result = await middleware_agent.run(prompt)
print(f"\n💬 Response: {result.text}")

# 7. Agent Memory

> **Why this matters:** Support agents should remember user preferences (language, tone, name) to provide personalized service across conversations.

## Define User Preferences Model

Store preferences that affect response generation:

In [ ]:
class SupportPreferences(BaseModel):
    """User preferences for support interactions."""
    name: str | None = None
    preferred_language: Literal["English", "Hebrew", "Spanish"] = "English"
    preferred_tone: Literal["formal", "friendly", "brief"] = "formal"

print("✅ SupportPreferences model defined")

## Implement ContextProvider

The `ContextProvider` has two key methods:
- `invoking`: Inject context before each agent call
- `invoked`: Update state after each call

In [ ]:
from collections.abc import MutableSequence, Sequence
from typing import Any
import re

from agent_framework import ContextProvider, Context, ChatAgent


class SupportMemory(ContextProvider):
    """Memory that tracks user preferences for support interactions."""
    
    def __init__(self, preferences: SupportPreferences | None = None):
        self.preferences = preferences or SupportPreferences()
    
    def _extract_name(self, text: str) -> str | None:
        """Extract name from text patterns."""
        patterns = [
            r"(?:my name is|i'm|i am|call me)\s+([A-Z][a-z]+)",
            r"(?:name is|name's)\s+([A-Z][a-z]+)",
        ]
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).capitalize()
        return None
    
    def _extract_preferences(self, text: str) -> None:
        """Extract tone and language preferences from text."""
        text_lower = text.lower()
        
        # Detect tone preferences
        if any(word in text_lower for word in ["friendly", "casual", "informal"]):
            self.preferences.preferred_tone = "friendly"
            print(f"   🧠 Memory updated: tone = friendly")
        elif any(word in text_lower for word in ["brief", "short", "concise"]):
            self.preferences.preferred_tone = "brief"
            print(f"   🧠 Memory updated: tone = brief")
        elif any(word in text_lower for word in ["formal", "professional"]):
            self.preferences.preferred_tone = "formal"
            print(f"   🧠 Memory updated: tone = formal")
        
        # Detect language preferences
        if "hebrew" in text_lower or "עברית" in text:
            self.preferences.preferred_language = "Hebrew"
            print(f"   🧠 Memory updated: language = Hebrew")
        elif "spanish" in text_lower or "español" in text_lower:
            self.preferences.preferred_language = "Spanish"
            print(f"   🧠 Memory updated: language = Spanish")
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Extract preferences from user messages after each call."""
        messages_list = [request_messages] if isinstance(request_messages, ChatMessage) else list(request_messages)
        
        for msg in messages_list:
            if msg.role.value == "user":
                text = ""
                if msg.contents:
                    for content in msg.contents:
                        if hasattr(content, 'text'):
                            text += content.text + " "
                
                # Extract name if not known
                if self.preferences.name is None:
                    name = self._extract_name(text)
                    if name:
                        self.preferences.name = name
                        print(f"   🧠 Memory updated: name = {name}")
                
                # Extract other preferences
                self._extract_preferences(text)
    
    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs: Any) -> Context:
        """Provide preference context before each agent call."""
        instructions = []
        
        if self.preferences.name:
            instructions.append(f"The user's name is {self.preferences.name}. Address them by name.")
        
        instructions.append(f"Respond in {self.preferences.preferred_language}.")
        instructions.append(f"Use a {self.preferences.preferred_tone} tone.")
        
        return Context(instructions=" ".join(instructions))
    
    def serialize(self) -> str:
        """Serialize for persistence."""
        return self.preferences.model_dump_json()

print("✅ SupportMemory ContextProvider defined")

In [ ]:
from typing import Callable, Awaitable
from agent_framework import AgentRunContext
import time

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Simple middleware that logs agent execution with timing."""
    # context.messages contains the input messages
    print(f"🚀 Agent starting... (messages: {len(context.messages)} message(s))")
    start_time = time.time()

    # Continue to agent execution
    await next(context)

    elapsed = time.time() - start_time
    print(f"✅ Agent finished! (took {elapsed:.2f}s)")

## Test Memory in Action

Watch how the agent adapts based on remembered preferences:

In [ ]:
# Create memory and agent
support_memory = SupportMemory()

memory_agent = ChatAgent(
    name="MemorySupportAgent",
    instructions="You are a friendly support agent. Adapt your responses based on user preferences.",
    chat_client=chat_client,
    context_providers=support_memory,
)

memory_thread = memory_agent.get_new_thread()

# Turn 1: User introduces themselves
print("Turn 1: User introduction")
print("-" * 50)
result1 = await memory_agent.run("Hi, my name is David", thread=memory_thread)
print(f"Agent: {result1.text}\n")

# Turn 2: User sets preference
print("Turn 2: Setting preference")
print("-" * 50)
result2 = await memory_agent.run("Please keep responses brief and casual", thread=memory_thread)
print(f"Agent: {result2.text}\n")

# Turn 3: Ask a question - agent should use name and brief/casual tone
print("Turn 3: Question with preferences applied")
print("-" * 50)
result3 = await memory_agent.run("What's your return policy?", thread=memory_thread)
print(f"Agent: {result3.text}\n")

# Show memory state
print("🧠 Memory State:")
print(f"   Name: {support_memory.preferences.name}")
print(f"   Language: {support_memory.preferences.preferred_language}")
print(f"   Tone: {support_memory.preferences.preferred_tone}")

# 8. Sequential Workflows

> **Why this matters:** Real support pipelines need multiple steps: classify → draft → review. Workflows orchestrate this flow with clear separation of concerns.

## Workflow Building Blocks

| Concept | Description |
|---------|-------------|
| **Executor** | A unit of work (class with `@handler` OR function with `@executor`) |
| **WorkflowBuilder** | Connects executors with `add_edge()` |
| `ctx.send_message()` | Pass data to next executor |
| `ctx.yield_output()` | Return final workflow result |

## Define Workflow Executors

Create executors for: Classify → Draft → Review

In [ ]:
from typing_extensions import Never
from agent_framework import (
    WorkflowBuilder, WorkflowContext, WorkflowOutputEvent,
    Executor, executor, handler, AgentExecutor, AgentExecutorRequest, AgentExecutorResponse
)

# === CLASSIFIER AGENT ===
classifier_agent = AgentExecutor(
    chat_client.as_agent(
        name="Classifier",
        instructions="""Classify incoming emails. Return JSON with:
- category: "spam", "not_spam", or "uncertain"
- confidence: float 0-1
- reason: brief explanation""",
        response_format=ClassificationResult,
    ),
    id="classifier",
)

# === DRAFT WRITER AGENT ===
writer_agent = AgentExecutor(
    chat_client.as_agent(
        name="DraftWriter",
        instructions="""Draft professional support responses. Return JSON with:
- subject: reply subject line
- body: reply body
- tone: "formal", "friendly", or "apologetic"
- needs_review: true if sensitive or complex""",
        response_format=DraftResponse,
    ),
    id="writer",
)

# === REVIEWER AGENT ===
reviewer_agent = AgentExecutor(
    chat_client.as_agent(
        name="Reviewer",
        instructions="""Review draft responses for quality. Check:
- Professionalism and tone
- Accuracy of information
- Completeness
Return approval decision with notes.""",
    ),
    id="reviewer",
)

print("✅ Workflow agents defined: classifier, writer, reviewer")

## Build and Run Sequential Workflow

Connect executors: Classifier → Writer → Reviewer

In [ ]:
# Build sequential workflow
sequential_support_workflow = (
    WorkflowBuilder()
    .set_start_executor(classifier_agent)
    .add_edge(classifier_agent, writer_agent)
    .add_edge(writer_agent, reviewer_agent)
    .build()
)

# Run with legitimate email
async def run_sequential_workflow():
    email_prompt = f"""Process this support email:

From: {LEGIT_EMAIL.sender}
Subject: {LEGIT_EMAIL.subject}
Customer ID: {LEGIT_EMAIL.customer_id}

{LEGIT_EMAIL.body}
"""
    
    print("📧 Processing email through workflow: Classify → Draft → Review\n")
    print("-" * 60)
    
    request = AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=email_prompt)],
        should_respond=True
    )
    
    from agent_framework._workflows._events import ExecutorCompletedEvent
    
    async for event in sequential_support_workflow.run_stream(request):
        if isinstance(event, ExecutorCompletedEvent) and event.data:
            data = event.data[0] if isinstance(event.data, list) else event.data
            if hasattr(data, 'agent_response'):
                print(f"\n✅ [{event.executor_id}]:")
                print(f"   {data.agent_response.text[:300]}...")
        elif isinstance(event, WorkflowOutputEvent):
            print(f"\n🎯 FINAL OUTPUT:")
            if isinstance(event.data, list) and event.data:
                final = event.data[0]
                if hasattr(final, 'agent_response'):
                    print(final.agent_response.text)

await run_sequential_workflow()

# 9. Branching Logic

> **Why this matters:** Different email types need different handling — spam should be blocked, uncertain emails need human review. Branching enables intelligent routing.

## Three Routing Patterns

| Pattern | Use Case | Targets |
|---------|----------|---------|
| **Conditional Edge** | Binary decision (if/else) | Exactly 1 |
| **Switch-Case** | Multi-way routing (enum) | Exactly 1 |
| **Multi-Selection** | Dynamic fan-out | 1 or more |

## Define Branching Executors

Handle Spam / NotSpam / Uncertain paths:

In [ ]:
from dataclasses import dataclass
from uuid import uuid4
from agent_framework import Case, Default

# Internal payload for routing
@dataclass
class ClassifiedEmail:
    email_id: str
    category: str  # spam, not_spam, uncertain
    confidence: float
    reason: str
    original_content: str

# Shared state keys
EMAIL_KEY = "current_email"

# Transform classification result to routable payload
@executor(id="extract_classification")
async def extract_classification(response: Any, ctx: WorkflowContext[ClassifiedEmail]) -> None:
    """Extract classification from agent response for routing."""
    if isinstance(response, list):
        response = response[0]
    
    classification = ClassificationResult.model_validate_json(response.agent_response.text)
    
    # Get original email from shared state
    original_content = await ctx.get_shared_state(EMAIL_KEY) or "Unknown"
    
    payload = ClassifiedEmail(
        email_id=str(uuid4()),
        category=classification.category,
        confidence=classification.confidence,
        reason=classification.reason,
        original_content=original_content
    )
    await ctx.send_message(payload)

# Route conditions
def is_spam(message: Any) -> bool:
    return isinstance(message, ClassifiedEmail) and message.category == "spam"

def is_not_spam(message: Any) -> bool:
    return isinstance(message, ClassifiedEmail) and message.category == "not_spam"

def is_uncertain(message: Any) -> bool:
    return isinstance(message, ClassifiedEmail) and message.category == "uncertain"

# Terminal handlers
@executor(id="handle_spam")
async def handle_spam_terminal(email: ClassifiedEmail, ctx: WorkflowContext[Never, str]) -> None:
    """Handle spam: block and log."""
    await ctx.yield_output(f"🚫 SPAM BLOCKED: {email.reason} (confidence: {email.confidence:.0%})")

@executor(id="handle_not_spam")
async def handle_not_spam_continue(email: ClassifiedEmail, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Handle not_spam: forward to writer."""
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Draft a response to: {email.original_content}")],
        should_respond=True
    ))

@executor(id="finalize_draft")
async def finalize_draft(response: Any, ctx: WorkflowContext[Never, str]) -> None:
    """Output the final draft."""
    if isinstance(response, list):
        response = response[0]
    draft = DraftResponse.model_validate_json(response.agent_response.text)
    await ctx.yield_output(f"✉️ DRAFT READY:\nSubject: {draft.subject}\n\n{draft.body}")

@executor(id="handle_uncertain")
async def handle_uncertain_terminal(email: ClassifiedEmail, ctx: WorkflowContext[Never, str]) -> None:
    """Handle uncertain: flag for human review."""
    await ctx.yield_output(f"⚠️ NEEDS HUMAN REVIEW: {email.reason} (confidence: {email.confidence:.0%})\n\nOriginal: {email.original_content[:200]}...")

print("✅ Branching executors defined")

## Build Branching Workflow

Use switch-case to route: Spam → Block, NotSpam → Draft, Uncertain → Review

In [ ]:
# Store email and start classification
@executor(id="start_classification")
async def start_classification(email_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Store email and send for classification."""
    await ctx.set_shared_state(EMAIL_KEY, email_text)
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Classify this email:\n\n{email_text}")],
        should_respond=True
    ))

# Build branching workflow
branching_workflow = (
    WorkflowBuilder()
    .set_start_executor(start_classification)
    .add_edge(start_classification, classifier_agent)
    .add_edge(classifier_agent, extract_classification)
    # Switch-case routing
    .add_switch_case_edge_group(
        extract_classification,
        [
            Case(condition=is_spam, target=handle_spam_terminal),
            Case(condition=is_not_spam, target=handle_not_spam_continue),
            Default(target=handle_uncertain_terminal),  # Catches uncertain + unexpected
        ],
    )
    # Continue not_spam path to draft
    .add_edge(handle_not_spam_continue, writer_agent)
    .add_edge(writer_agent, finalize_draft)
    .build()
)

print("✅ Branching workflow built")

## Test All Branches

Run all three email types to see different paths:

In [ ]:
# Test all three paths
async def test_branching():
    test_cases = [
        ("LEGITIMATE", LEGIT_EMAIL),
        ("SPAM", SPAM_EMAIL),
        ("AMBIGUOUS", AMBIGUOUS_EMAIL),
    ]
    
    for label, email in test_cases:
        print(f"\n📧 Testing {label} email...")
        print("-" * 50)
        
        email_text = f"From: {email.sender}\nSubject: {email.subject}\n\n{email.body}"
        
        async for event in branching_workflow.run_stream(email_text):
            if isinstance(event, WorkflowOutputEvent):
                print(event.data)

await test_branching()

# 10. Fan-Out / Fan-In

> **Why this matters:** Long emails benefit from parallel processing — draft a response AND create a summary simultaneously, then combine results.

## Define Parallel Processing Executors

For long emails: respond AND summarize in parallel

In [ ]:
# Summary model
class EmailSummary(BaseModel):
    """Concise email summary."""
    key_points: list[str] = Field(description="Main points from the email")
    urgency: Literal["low", "medium", "high"] = Field(description="Urgency level")
    action_required: str = Field(description="Primary action needed")

# Summarizer agent
summarizer_agent = AgentExecutor(
    chat_client.as_agent(
        name="Summarizer",
        instructions="""Summarize emails concisely. Return JSON with:
- key_points: list of main points
- urgency: low/medium/high
- action_required: primary action needed""",
        response_format=EmailSummary,
    ),
    id="summarizer",
)

# Threshold for "long" emails
LONG_EMAIL_THRESHOLD = 200  # characters

@dataclass
class EnrichedEmail:
    """Email with metadata for routing."""
    email_id: str
    content: str
    is_long: bool
    category: str

# Selection function for multi-selection routing
def select_parallel_paths(email: EnrichedEmail, target_ids: list[str]) -> list[str]:
    """Select paths based on email length."""
    # target_ids order: [respond_path, summarize_path]
    respond_id, summarize_id = target_ids
    
    if email.is_long:
        return [respond_id, summarize_id]  # Both paths in parallel
    else:
        return [respond_id]  # Only respond for short emails

# Executors for parallel paths
@executor(id="prepare_parallel")
async def prepare_parallel(classified: ClassifiedEmail, ctx: WorkflowContext[EnrichedEmail]) -> None:
    """Prepare email for parallel processing."""
    enriched = EnrichedEmail(
        email_id=classified.email_id,
        content=classified.original_content,
        is_long=len(classified.original_content) > LONG_EMAIL_THRESHOLD,
        category=classified.category
    )
    await ctx.send_message(enriched)

@executor(id="respond_path")
async def respond_path(email: EnrichedEmail, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Send to writer for response."""
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Draft a response to:\n{email.content}")],
        should_respond=True
    ))

@executor(id="summarize_path")
async def summarize_path(email: EnrichedEmail, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Send to summarizer."""
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Summarize this email:\n{email.content}")],
        should_respond=True
    ))

# Aggregator to combine parallel results
class ParallelAggregator(Executor):
    def __init__(self):
        super().__init__(id="parallel_aggregator")
    
    @handler
    async def aggregate(self, results: list[Any], ctx: WorkflowContext[Never, str]) -> None:
        """Combine response and summary."""
        output_parts = []
        
        for result in results:
            if isinstance(result, AgentExecutorResponse):
                try:
                    draft = DraftResponse.model_validate_json(result.agent_response.text)
                    output_parts.append(f"📧 DRAFT RESPONSE:\nSubject: {draft.subject}\n{draft.body}")
                except:
                    try:
                        summary = EmailSummary.model_validate_json(result.agent_response.text)
                        points = "\n".join(f"  • {p}" for p in summary.key_points)
                        output_parts.append(f"📋 SUMMARY:\n{points}\nUrgency: {summary.urgency}\nAction: {summary.action_required}")
                    except:
                        output_parts.append(f"Result: {result.agent_response.text[:200]}...")
        
        await ctx.yield_output("\n\n" + "="*40 + "\n\n".join(output_parts))

aggregator = ParallelAggregator()

print("✅ Parallel processing executors defined")

## Build Fan-Out/Fan-In Workflow

Short emails → respond only. Long emails → respond + summarize in parallel.

In [ ]:
from agent_framework import WorkflowBuilder
from agent_framework._workflows._events import ExecutorCompletedEvent

# Constants
LONG_EMAIL_THRESHOLD = 200  # Characters

# Start executor - entry point stores email and passes it forward
@executor(id="fanout_start")
async def fanout_start(email_text: str, ctx: WorkflowContext[str]) -> None:
    """Entry point: store email length, forward email text."""
    # Store email length in shared state for selection
    await ctx.set_shared_state("email_length", len(email_text))
    await ctx.send_message(email_text)

# Selection function that uses shared state
def fanout_select_paths(email_text: str, target_ids: list[str]) -> list[str]:
    """Select paths based on email length (stored in text)."""
    # The email_text is still the raw string at this point
    if len(email_text) > LONG_EMAIL_THRESHOLD:
        return target_ids  # Both paths for long emails
    return [target_ids[0]]  # Only response path for short emails

# Response path preparer
@executor(id="fanout_respond_prep")
async def fanout_respond_prep(email_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Prepare email for writer agent."""
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Draft a response to:\n{email_text}")],
        should_respond=True
    ))

# Summary path preparer
@executor(id="fanout_summarize_prep")
async def fanout_summarize_prep(email_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Prepare email for summarizer agent."""
    await ctx.send_message(AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=f"Summarize this email:\n{email_text}")],
        should_respond=True
    ))

# Aggregator - combines results from parallel paths
@executor(id="fanout_aggregator")
async def fanout_aggregator(results: list[Any], ctx: WorkflowContext[Never, str]) -> None:
    """Combine response and summary results."""
    output_parts = []
    for result in results:
        if isinstance(result, AgentExecutorResponse):
            try:
                draft = DraftResponse.model_validate_json(result.agent_response.text)
                output_parts.append(f"📬 RESPONSE:\nSubject: {draft.subject}\n{draft.body}")
            except:
                try:
                    summary = EmailSummary.model_validate_json(result.agent_response.text)
                    points = "\n".join(f"  • {p}" for p in summary.key_points)
                    output_parts.append(f"📋 SUMMARY:\n{points}\nUrgency: {summary.urgency}\nAction: {summary.action_required}")
                except:
                    output_parts.append(f"Result: {result.agent_response.text[:200]}...")
    
    await ctx.yield_output("\n\n" + "="*50 + "\n\n".join(output_parts))

# Build the fan-out workflow
# Pattern: start -> [fanout to preparers] -> [agents] -> aggregator
fanout_workflow = (
    WorkflowBuilder()
    .set_start_executor(fanout_start)
    # Fan-out from start directly to path preparers based on email length
    .add_multi_selection_edge_group(
        fanout_start,
        targets=[fanout_respond_prep, fanout_summarize_prep],
        selection_func=fanout_select_paths,
    )
    # Each preparer sends to its agent
    .add_edge(fanout_respond_prep, writer_agent)
    .add_edge(fanout_summarize_prep, summarizer_agent)
    # Fan-in: collect all results
    .add_fan_in_edges([writer_agent, summarizer_agent], fanout_aggregator)
    .build()
)

print("✅ Fan-out/fan-in workflow built")

## Test Parallel Processing

Compare short vs long email processing:

In [ ]:
# Test with long legitimate email
async def test_fanout():
    email_text = f"From: {LEGIT_EMAIL.sender}\nSubject: {LEGIT_EMAIL.subject}\n\n{LEGIT_EMAIL.body}"
    
    print(f"📧 Testing LONG email ({len(email_text)} chars > {LONG_EMAIL_THRESHOLD} threshold)")
    print("Expected: Response AND Summary in parallel\n")
    print("-" * 60)
    
    async for event in fanout_workflow.run_stream(email_text):
        if isinstance(event, WorkflowOutputEvent):
            print(event.data)

await test_fanout()

# 11. Multi-Agent Group Chat

> **Why this matters:** Complex quality control needs multiple perspectives — security, tone, accuracy. Group chat enables collaborative review.

## Group Chat Patterns

| Pattern | Builder | How it Works |
|---------|---------|--------------|
| **Sequential** | `SequentialBuilder` | Agents take turns (round-robin) |
| **Concurrent** | `ConcurrentBuilder` | All agents process in parallel |
| **Magentic** | `MagenticBuilder` | Manager orchestrates who speaks |

## Create Reviewer Team (Concurrent)

Three reviewers analyze drafts in parallel:

In [ ]:
from agent_framework import ConcurrentBuilder, SequentialBuilder, MagenticBuilder

# Three specialized reviewers
security_reviewer = ChatAgent(
    name="SecurityReviewer",
    description="Checks for security and compliance issues",
    instructions="Review support responses for security issues: no sensitive data exposed, no phishing risks, compliance with policies. List top concerns. Be brief.",
    chat_client=chat_client,
)

tone_reviewer = ChatAgent(
    name="ToneReviewer",
    description="Checks tone and empathy",
    instructions="Review support responses for appropriate tone: professional, empathetic, not defensive. List suggestions. Be brief.",
    chat_client=chat_client,
)

accuracy_reviewer = ChatAgent(
    name="AccuracyReviewer",
    description="Checks factual accuracy",
    instructions="Review support responses for accuracy: no false promises, realistic timelines, correct information. List concerns. Be brief.",
    chat_client=chat_client,
)

# Build concurrent review team
review_team = (
    ConcurrentBuilder()
    .participants([security_reviewer, tone_reviewer, accuracy_reviewer])
    .build()
)

print("✅ Concurrent review team created: Security || Tone || Accuracy")

## Test Concurrent Review

All three reviewers analyze the draft simultaneously:

In [ ]:
# Sample draft response to review
draft_to_review = """
Subject: Re: Order #12345 - Delivery Issue

Dear Sarah,

I'm so sorry to hear about the missing package! This must be incredibly frustrating.

I've located your order and can confirm it was marked as delivered on Monday. Here's what I'll do:

1. I've opened an investigation with our shipping partner (Case #INV-789)
2. As a Premium customer, I'm expediting a replacement shipment TODAY
3. The replacement will arrive by Thursday, well before your Friday presentation

Your account has also been credited $50 for the inconvenience.

If you need anything else, reply directly to this email - I'm here to help!

Best regards,
Support Team
"""

async def test_concurrent_review():
    print("📝 Draft to review:")
    print(draft_to_review)
    print("-" * 60)
    print("\n🔍 PARALLEL REVIEWS (all running simultaneously):\n")
    
    async for event in review_team.run_stream(f"Review this support response:\n{draft_to_review}"):
        if isinstance(event, WorkflowOutputEvent):
            messages = event.data
            for msg in messages:
                if msg.role.value == "assistant":
                    print(f"\n{msg.text}")
                    print("-" * 40)

await test_concurrent_review()

## Magentic Team (Manager-Orchestrated)

For complex cases, a manager decides which specialists to involve:

In [ ]:
# Manager agent for orchestrated review
review_manager = ChatAgent(
    name="ReviewManager",
    description="Coordinates the review process",
    instructions="""You manage a team reviewing support responses. 
Delegate to specialists based on the content:
- SecurityReviewer for compliance/data concerns
- ToneReviewer for customer experience
- AccuracyReviewer for factual correctness

Synthesize feedback into a final approval decision.""",
    chat_client=chat_client,
)

# Magentic team with manager
magentic_review_team = (
    MagenticBuilder()
    .participants([security_reviewer, tone_reviewer, accuracy_reviewer])
    .with_manager(
        agent=review_manager,
        max_round_count=6,
        max_stall_count=2,
    )
    .build()
)

print("✅ Magentic review team created: Manager → [Security, Tone, Accuracy]")

In [ ]:
# Test magentic review team
from typing import cast

async def test_magentic_review():
    print("🎯 Manager-orchestrated review of draft response:\n")
    print("-" * 60)
    
    output = None
    async for event in magentic_review_team.run_stream(f"Review this support response and provide final approval decision:\n{draft_to_review}"):
        if isinstance(event, WorkflowOutputEvent):
            output_messages = cast(list[ChatMessage], event.data)
            if output_messages:
                output = output_messages[-1].text
    
    print("\n📋 FINAL REVIEW DECISION:")
    print(output)

await test_magentic_review()

# 12. Capstone Demo

> **Putting it all together:** Run the complete Support Email Copilot system end-to-end on both a legitimate and spam email.

## Complete Support Email Pipeline

This demo shows the full flow:
1. **Classify** the email (Spam/NotSpam/Uncertain)
2. **Route** based on classification
3. **Lookup** customer info (for legitimate emails)
4. **Draft** a response
5. **Review** with multi-agent team
6. **Output** final result

In [ ]:
async def run_capstone_demo():
    """Complete end-to-end demo of the Support Email Copilot."""
    
    print("=" * 70)
    print("🚀 SUPPORT EMAIL COPILOT - CAPSTONE DEMO")
    print("=" * 70)
    
    test_emails = [
        ("LEGITIMATE SUPPORT REQUEST", LEGIT_EMAIL),
        ("SPAM", SPAM_EMAIL),
    ]
    
    for label, email in test_emails:
        print(f"\n\n{'='*70}")
        print(f"📧 PROCESSING: {label}")
        print(f"{'='*70}")
        print(f"From: {email.sender}")
        print(f"Subject: {email.subject}")
        print(f"Customer ID: {email.customer_id}")
        print("-" * 50)
        
        email_text = f"From: {email.sender}\nSubject: {email.subject}\n\n{email.body}"
        
        # Step 1: Classification
        print("\n📊 Step 1: CLASSIFICATION")
        classification_result = await classifier_agent.agent.run(
            f"Classify this email:\n\n{email_text}"
        )
        classification = ClassificationResult.model_validate_json(classification_result.text)
        print(f"   Category: {classification.category}")
        print(f"   Confidence: {classification.confidence:.0%}")
        print(f"   Reason: {classification.reason}")
        
        # Step 2: Route based on classification
        if classification.category == "spam":
            print("\n🚫 Step 2: ROUTED TO SPAM HANDLER")
            print("   Result: Email blocked and logged")
            continue
        
        if classification.category == "uncertain":
            print("\n⚠️ Step 2: ROUTED TO HUMAN REVIEW")
            print("   Result: Flagged for manual review")
            continue
        
        # Step 3: For legitimate emails - lookup customer info
        print("\n🔍 Step 3: CUSTOMER LOOKUP")
        if email.customer_id:
            sla_info = lookup_customer_sla(email.customer_id)
            print(f"   SLA: {sla_info}")
        if email.ticket_id:
            ticket_info = get_incident_status(email.ticket_id)
            print(f"   Ticket: {ticket_info}")
        
        # Step 4: Draft response
        print("\n✍️ Step 4: DRAFTING RESPONSE")
        draft_result = await writer_agent.agent.run(
            f"Draft a professional support response for this email. Customer is Premium tier.\n\n{email_text}"
        )
        draft = DraftResponse.model_validate_json(draft_result.text)
        print(f"   Subject: {draft.subject}")
        print(f"   Tone: {draft.tone}")
        print(f"   Body preview: {draft.body[:200]}...")
        
        # Step 5: Concurrent review
        print("\n🔍 Step 5: MULTI-AGENT REVIEW (parallel)")
        review_results = []
        async for event in review_team.run_stream(f"Review this draft:\n{draft.body}"):
            if isinstance(event, WorkflowOutputEvent):
                messages = event.data
                for msg in messages:
                    if msg.role.value == "assistant":
                        review_results.append(msg.text[:150])
        
        for i, review in enumerate(review_results[:3], 1):
            print(f"   Reviewer {i}: {review}...")
        
        # Step 6: Final output
        print("\n✅ Step 6: FINAL OUTPUT")
        print(f"   Status: Ready for approval")
        print(f"   Draft approved for sending to: {email.sender}")
    
    print("\n\n" + "=" * 70)
    print("🎉 CAPSTONE DEMO COMPLETE")
    print("=" * 70)
    print("\nYou've seen the complete Support Email Copilot system:")
    print("• Classification with routing (Spam/NotSpam/Uncertain)")
    print("• Customer lookup via function tools")
    print("• Response drafting with structured output")
    print("• Multi-agent concurrent review")
    print("• End-to-end processing pipeline")

await run_capstone_demo()